In [1]:
import sqlite3 as sql
import pandas as pd

In [8]:
conn = sql.connect("Assignment2_mod.db")
#conn.execute("PRAGMA foreign_keys = ON")

try:
    # Create tables, dropping them if they exist before the creation.
    #Projects table
    conn.execute("""
    DROP TABLE IF EXISTS projects
    """)    
    conn.execute("""
    CREATE TABLE projects(
        id INT NOT NULL,
        name TEXT,
        status TEXT,
        start_month TEXT,
        start_year INT,
        PRIMARY KEY (id),
        UNIQUE(id))
    """)
    
    #Researchers table
    conn.execute("""
    DROP TABLE IF EXISTS researchers
    """)
    conn.execute("""
    CREATE TABLE researchers(
        id INT NOT NULL,
        first_name TEXT,
        last_name TEXT,
        email TEXT,
        phone TEXT,
        department_id INT,
        PRIMARY KEY (id),
        FOREIGN KEY (department_id) REFERENCES departments(id) ON DELETE CASCADE ON UPDATE CASCADE)
    """)
    
    #Departments table
    conn.execute("""
    DROP TABLE IF EXISTS departments
    """)
    conn.execute("""
    CREATE TABLE departments(
        id INT NOT NULL,
        name TEXT NOT NULL,
        campus TEXT,
        PRIMARY KEY (id))
    """)
    
    #Tasks table
    conn.execute("""
    DROP TABLE IF EXISTS tasks
    """)
    conn.execute("""
    CREATE TABLE tasks(
        id INT NOT NULL,
        name TEXT,
        status TEXT,
        project_id INT,
        researcher_id INT,
        PRIMARY KEY (id),
        FOREIGN KEY (project_id) REFERENCES projects(id) ON DELETE CASCADE ON UPDATE CASCADE,
        FOREIGN KEY (researcher_id) REFERENCES researchers(id) ON DELETE CASCADE ON UPDATE CASCADE)
    """)
    
    #Equipment table
    conn.execute("""
    DROP TABLE IF EXISTS equipment
    """)
    conn.execute("""
    CREATE TABLE equipment(
        id INT NOT NULL,
        type TEXT NOT NULL,
        brand TEXT,
        model TEXT,
        PRIMARY KEY (id))
    """)
    
    # Create linking tables for M-to-N relationships; drop them if they exist before the creation. 
    
    #Researcher-Project table
    conn.execute("""
    DROP TABLE IF EXISTS researcher_project
    """)    
    conn.execute("""
    CREATE TABLE researcher_project(
        researcher_id INT,
        project_id INT,
        PRIMARY KEY (researcher_id, project_id),
        FOREIGN KEY (researcher_id) REFERENCES researchers(id) ON DELETE CASCADE ON UPDATE CASCADE,
        FOREIGN KEY (project_id) REFERENCES projects(id) ON DELETE CASCADE ON UPDATE CASCADE)
    """)
    
    #Researcher-equipment table
    conn.execute("""
    DROP TABLE IF EXISTS researcher_equipment
    """)
    conn.execute("""
    CREATE TABLE researcher_equipment(
        researcher_id INT,
        equipment_id INT,
        PRIMARY KEY (researcher_id, equipment_id),
        FOREIGN KEY (researcher_id) REFERENCES researchers(id) ON DELETE CASCADE ON UPDATE CASCADE,
        FOREIGN KEY (equipment_id) REFERENCES equipment(id) ON DELETE CASCADE ON UPDATE CASCADE)
    """)
    
    #Task-Equipment table
    conn.execute("""
    DROP TABLE IF EXISTS task_equipment
    """)
    conn.execute("""
    CREATE TABLE task_equipment(
        task_id INT,
        equipment_id INT,
        PRIMARY KEY (task_id, equipment_id),
        FOREIGN KEY (task_id) REFERENCES tasks(id) ON DELETE CASCADE ON UPDATE CASCADE,
        FOREIGN KEY (equipment_id) REFERENCES equipment(id) ON DELETE CASCADE ON UPDATE CASCADE)
    """)
    
    #Add data to main tables
    conn.execute("INSERT INTO projects VALUES (0,'Project0','Active','January',2020)")
    conn.executemany("""
    INSERT INTO departments VALUES (?,?,?)""",[(1,'Engineering','Athens'),(2,'','Athens'),(3,'Plant production','Tifton'),(4,'Genetics','Riverbend')])
    conn.execute("INSERT INTO researchers VALUES (0,'Javier','Rodriguez','javier.rodriguez@uga.edu','686.300.9440',0)")
    conn.executemany("""
    INSERT INTO tasks VALUES (?,?,?,?,?)""",[(1,'Task1','Finished',1,2),(2,'Task2','Active',1,1),(3,'Task3','Active',2,2),(4,'Task4','Active',2,3)])
    conn.executemany("""
    INSERT INTO equipment VALUES (?,?,?,?)""",[(1,'Sensor','Panasonic','Lumix G7'),(2,'Multimeter','Fluke','FL-210'),(3,'Power Tool','Bosch','BS-550T'),(4,'Sensor','Panasonic','Lumix G7')])
    
    #Add data to link tables
    conn.executemany("""
    INSERT INTO researcher_project VALUES (?,?)""",[(1,1),(1,2),(2,1),(2,3),(3,2),(4,4)])
    conn.executemany("""
    INSERT INTO researcher_equipment VALUES (?,?)""",[(1,1),(1,2),(2,4),(2,2),(3,1),(4,1),(4,3)])
    conn.executemany("""
    INSERT INTO task_equipment VALUES (?,?)""",[(1,1),(1,2),(1,3),(2,4),(2,2),(3,2),(3,3),(4,4),(4,2)])
    
    
    
    df = pd.read_sql_query("select * from projects;", conn)
    print("Projects\n", df)
    df = pd.read_sql_query("select * from researchers;", conn)
    print("Researchers\n", df)
    df = pd.read_sql_query("select * from departments;", conn)
    print("Departments\n", df)
    df = pd.read_sql_query("select * from tasks;", conn)
    print("Tasks\n", df)
    df = pd.read_sql_query("select * from equipment;", conn)
    print("Equipment\n", df)
    
    df = pd.read_sql_query("select * from researcher_project;", conn)
    print("Researcher-Project\n", df)
    df = pd.read_sql_query("select * from researcher_equipment;", conn)
    print("Researcher-Equipment\n", df)
    df = pd.read_sql_query("select * from task_equipment;", conn)
    print("Task-Equipment\n", df)
    
    conn.commit();
    
except sql.Error as e:
    print("There was error:" + str(e))
    
finally:
    print("this executed")
    #conn.close()

Projects
    id      name  status start_month  start_year
0   0  Project0  Active     January        2020
Researchers
    id first_name  last_name                     email         phone  \
0   0     Javier  Rodriguez  javier.rodriguez@uga.edu  686.300.9440   

   department_id  
0              0  
Departments
    id              name     campus
0   1       Engineering     Athens
1   2                       Athens
2   3  Plant production     Tifton
3   4          Genetics  Riverbend
Tasks
    id   name    status  project_id  researcher_id
0   1  Task1  Finished           1              2
1   2  Task2    Active           1              1
2   3  Task3    Active           2              2
3   4  Task4    Active           2              3
Equipment
    id        type      brand     model
0   1      Sensor  Panasonic  Lumix G7
1   2  Multimeter      Fluke    FL-210
2   3  Power Tool      Bosch   BS-550T
3   4      Sensor  Panasonic  Lumix G7
Researcher-Project
    researcher_id  project_id


First we insert data in tables

In [9]:
conn.execute("INSERT INTO researchers (id,first_name,last_name,email,phone,department_id) VALUES (107,'Summer','Payne','summer.payne@uga.edu','515.123.8181',106)")
conn.execute("INSERT INTO researchers (id,first_name,last_name,email,phone,department_id) VALUES (106,'Rose','Stephens','rose.stephens@uga.edu','515.123.8080',2)")
conn.execute("INSERT INTO researchers (id,first_name,last_name,email,phone,department_id) VALUES (101,'Annabelle','Dunn','annabelle.dunn@uga.edu','515.123.4444',2)")
conn.execute("INSERT INTO researchers (id,first_name,last_name,email,phone,department_id) VALUES (1,'Tommy','Bailey','tommy.bailey@uga.edu','515.123.4567',null)")
conn.execute("INSERT INTO researchers (id,first_name,last_name,email,phone,department_id) VALUES (3,'Blake','Cooper','blake.cooper@uga.edu','515.123.4569',1)")
conn.execute("INSERT INTO researchers (id,first_name,last_name,email,phone,department_id) VALUES (2,'Jude','Rivera','jude.rivera@uga.edu','515.123.4568',1)")
conn.execute("INSERT INTO researchers (id,first_name,last_name,email,phone,department_id) VALUES (11,'Tyler','Ramirez','tyler.ramirez@uga.edu','515.124.4269',9)")
conn.execute("INSERT INTO researchers (id,first_name,last_name,email,phone,department_id) VALUES (10,'Ryan','Gray','ryan.gray@uga.edu','515.124.4169',9)")
conn.execute("INSERT INTO researchers (id,first_name,last_name,email,phone,department_id) VALUES (14,'Elliot','Brooks','elliot.brooks@uga.edu','515.124.4567',9)")
conn.execute("INSERT INTO researchers (id,first_name,last_name,email,phone,department_id) VALUES (12,'Elliott','James','elliott.james@uga.edu','515.124.4369',9)")
conn.execute("INSERT INTO researchers (id,first_name,last_name,email,phone,department_id) VALUES (13,'Albert','Watson','albert.watson@uga.edu','515.124.4469',9)")
conn.execute("INSERT INTO researchers (id,first_name,last_name,email,phone,department_id) VALUES (9,'Mohammad','Peterson','mohammad.peterson@uga.edu','515.124.4569',2)")
conn.execute("INSERT INTO researchers (id,first_name,last_name,email,phone,department_id) VALUES (104,'Harper','Spencer','harper.spencer@uga.edu','515.123.7777',2)")
conn.execute("INSERT INTO researchers (id,first_name,last_name,email,phone,department_id) VALUES (4,'Louie','Richardson','louie.richardson@uga.edu','590.423.4567',3)")
conn.execute("INSERT INTO researchers (id,first_name,last_name,email,phone,department_id) VALUES (5,'Nathan','Cox','nathan.cox@uga.edu','590.423.4568',4)")
conn.execute("INSERT INTO researchers (id,first_name,last_name,email,phone,department_id) VALUES (8,'Bobby','Torres','bobby.torres@uga.edu','590.423.5567',4)")
conn.execute("INSERT INTO researchers (id,first_name,last_name,email,phone,department_id) VALUES (7,'Charles','Ward','charles.ward@uga.edu','590.423.4560',4)")
conn.execute("INSERT INTO researchers (id,first_name,last_name,email,phone,department_id) VALUES (6,'Gabriel','Howard','gabriel.howard@uga.edu','590.423.4569',4)")
conn.execute("INSERT INTO researchers (id,first_name,last_name,email,phone,department_id) VALUES (102,'Emma','Perkins','emma.perkins@uga.edu','515.123.5555',1)")
conn.execute("INSERT INTO researchers (id,first_name,last_name,email,phone,department_id) VALUES (103,'Amelie','Hudson','amelie.hudson@uga.edu','603.123.6666',102)")
conn.execute("INSERT INTO researchers (id,first_name,last_name,email,phone,department_id) VALUES (105,'Gracie','Gardner','gracie.gardner@uga.edu','515.123.8888',2)")
conn.execute("INSERT INTO researchers (id,first_name,last_name,email,phone,department_id) VALUES (17,'Frederick','Price','frederick.price@uga.edu','515.127.4563',15)")
conn.execute("INSERT INTO researchers (id,first_name,last_name,email,phone,department_id) VALUES (16,'Alex','Sanders','alex.sanders@uga.edu','515.127.4562',15)")
conn.execute("INSERT INTO researchers (id,first_name,last_name,email,phone,department_id) VALUES (18,'Ollie','Bennett','ollie.bennett@uga.edu','515.127.4564',15)")
conn.execute("INSERT INTO researchers (id,first_name,last_name,email,phone,department_id) VALUES (19,'Louis','Wood','louis.wood@uga.edu','515.127.4565',15)")
conn.execute("INSERT INTO researchers (id,first_name,last_name,email,phone,department_id) VALUES (20,'Dexter','Barnes','dexter.barnes@uga.edu','515.127.4566',15)")
conn.execute("INSERT INTO researchers (id,first_name,last_name,email,phone,department_id) VALUES (15,'Rory','Kelly','rory.kelly@uga.edu','515.127.4561',1)")
conn.execute("INSERT INTO researchers (id,first_name,last_name,email,phone,department_id) VALUES (49,'Isabella','Cole','isabella.cole@uga.edu','011.44.1344.619268',1)")
conn.execute("INSERT INTO researchers (id,first_name,last_name,email,phone,department_id) VALUES (48,'Jessica','Woods','jessica.woods@uga.edu','011.44.1344.429278',1)")
conn.execute("INSERT INTO researchers (id,first_name,last_name,email,phone,department_id) VALUES (47,'Ella','Wallace','ella.wallace@uga.edu','011.44.1344.467268',1)")
conn.execute("INSERT INTO researchers (id,first_name,last_name,email,phone,department_id) VALUES (46,'Ava','Sullivan','ava.sullivan@uga.edu','011.44.1344.429268',1)")
conn.execute("INSERT INTO researchers (id,first_name,last_name,email,phone,department_id) VALUES (50,'Mia','West','mia.west@uga.edu','011.44.1344.429018',1)")
conn.execute("INSERT INTO researchers (id,first_name,last_name,email,phone,department_id) VALUES (56,'Evie','Harrison','evie.harrison@uga.edu','011.44.1344.486508',46)")
conn.execute("INSERT INTO researchers (id,first_name,last_name,email,phone,department_id) VALUES (57,'Scarlett','Gibson','scarlett.gibson@uga.edu','011.44.1345.429268',47)")
conn.execute("INSERT INTO researchers (id,first_name,last_name,email,phone,department_id) VALUES (58,'Ruby','Mcdonald','ruby.mcdonald@uga.edu','011.44.1345.929268',47)")
conn.execute("INSERT INTO researchers (id,first_name,last_name,email,phone,department_id) VALUES (59,'Chloe','Cruz','chloe.cruz@uga.edu','011.44.1345.829268',47)")
conn.execute("INSERT INTO researchers (id,first_name,last_name,email,phone,department_id) VALUES (60,'Isabelle','Marshall','isabelle.marshall@uga.edu','011.44.1345.729268',47)")
conn.execute("INSERT INTO researchers (id,first_name,last_name,email,phone,department_id) VALUES (61,'Daisy','Ortiz','daisy.ortiz@uga.edu','011.44.1345.629268',47)")
conn.execute("INSERT INTO researchers (id,first_name,last_name,email,phone,department_id) VALUES (62,'Freya','Gomez','freya.gomez@uga.edu','011.44.1345.529268',47)")
conn.execute("INSERT INTO researchers (id,first_name,last_name,email,phone,department_id) VALUES (80,'Elizabeth','Dixon','elizabeth.dixon@uga.edu','011.44.1644.429262',50)")
conn.execute("INSERT INTO researchers (id,first_name,last_name,email,phone,department_id) VALUES (64,'Florence','Freeman','florence.freeman@uga.edu','011.44.1346.229268',48)")
conn.execute("INSERT INTO researchers (id,first_name,last_name,email,phone,department_id) VALUES (65,'Alice','Wells','alice.wells@uga.edu','011.44.1346.329268',48)")
conn.execute("INSERT INTO researchers (id,first_name,last_name,email,phone,department_id) VALUES (66,'Charlotte','Webb','charlotte.webb@uga.edu','011.44.1346.529268',48)")
conn.execute("INSERT INTO researchers (id,first_name,last_name,email,phone,department_id) VALUES (67,'Sienna','Simpson','sienna.simpson@uga.edu','011.44.1346.629268',48)")
conn.execute("INSERT INTO researchers (id,first_name,last_name,email,phone,department_id) VALUES (68,'Matilda','Stevens','matilda.stevens@uga.edu','011.44.1346.729268',48)")
conn.execute("INSERT INTO researchers (id,first_name,last_name,email,phone,department_id) VALUES (69,'Evelyn','Tucker','evelyn.tucker@uga.edu','011.44.1343.929268',49)")
conn.execute("INSERT INTO researchers (id,first_name,last_name,email,phone,department_id) VALUES (70,'Eva','Porter','eva.porter@uga.edu','011.44.1343.829268',49)")
conn.execute("INSERT INTO researchers (id,first_name,last_name,email,phone,department_id) VALUES (71,'Millie','Hunter','millie.hunter@uga.edu','011.44.1343.729268',49)")
conn.execute("INSERT INTO researchers (id,first_name,last_name,email,phone,department_id) VALUES (72,'Sofia','Hicks','sofia.hicks@uga.edu','011.44.1343.629268',49)")
conn.execute("INSERT INTO researchers (id,first_name,last_name,email,phone,department_id) VALUES (73,'Lucy','Crawford','lucy.crawford@uga.edu','011.44.1343.529268',49)")
conn.execute("INSERT INTO researchers (id,first_name,last_name,email,phone,department_id) VALUES (74,'Elsie','Henry','elsie.henry@uga.edu','011.44.1343.329268',49)")
conn.execute("INSERT INTO researchers (id,first_name,last_name,email,phone,department_id) VALUES (75,'Imogen','Boyd','imogen.boyd@uga.edu','011.44.1644.429267',50)")
conn.execute("INSERT INTO researchers (id,first_name,last_name,email,phone,department_id) VALUES (76,'Layla','Mason','layla.mason@uga.edu','011.44.1644.429266',50)")
conn.execute("INSERT INTO researchers (id,first_name,last_name,email,phone,department_id) VALUES (77,'Rosie','Morales','rosie.morales@uga.edu','011.44.1644.429265',50)")
conn.execute("INSERT INTO researchers (id,first_name,last_name,email,phone,department_id) VALUES (78,'Maya','Kennedy','maya.kennedy@uga.edu','011.44.1644.429264',50)")
conn.execute("INSERT INTO researchers (id,first_name,last_name,email,phone,department_id) VALUES (79,'Esme','Warren','esme.warren@uga.edu','011.44.1644.429263',50)")
conn.execute("INSERT INTO researchers (id,first_name,last_name,email,phone,department_id) VALUES (55,'Grace','Ellis','grace.ellis@uga.edu','011.44.1344.987668',46)")
conn.execute("INSERT INTO researchers (id,first_name,last_name,email,phone,department_id) VALUES (54,'Lily','Fisher','lily.fisher@uga.edu','011.44.1344.498718',46)")
conn.execute("INSERT INTO researchers (id,first_name,last_name,email,phone,department_id) VALUES (53,'Sophia','Reynolds','sophia.reynolds@uga.edu','011.44.1344.478968',46)")
conn.execute("INSERT INTO researchers (id,first_name,last_name,email,phone,department_id) VALUES (52,'Sophie','Owens','sophie.owens@uga.edu','011.44.1344.345268',46)")
conn.execute("INSERT INTO researchers (id,first_name,last_name,email,phone,department_id) VALUES (51,'Poppy','Jordan','poppy.jordan@uga.edu','011.44.1344.129268',46)")
conn.execute("INSERT INTO researchers (id,first_name,last_name,email,phone,department_id) VALUES (63,'Phoebe','Murray','phoebe.murray@uga.edu','011.44.1346.129268',48)")
conn.execute("INSERT INTO researchers (id,first_name,last_name,email,phone,department_id) VALUES (85,'Holly','Shaw','holly.shaw@uga.edu','650.509.1876',22)")
conn.execute("INSERT INTO researchers (id,first_name,last_name,email,phone,department_id) VALUES (86,'Emilia','Holmes','emilia.holmes@uga.edu','650.509.2876',22)")
conn.execute("INSERT INTO researchers (id,first_name,last_name,email,phone,department_id) VALUES (87,'Molly','Rice','molly.rice@uga.edu','650.509.3876',22)")
conn.execute("INSERT INTO researchers (id,first_name,last_name,email,phone,department_id) VALUES (88,'Ellie','Robertson','ellie.robertson@uga.edu','650.509.4876',22)")
conn.execute("INSERT INTO researchers (id,first_name,last_name,email,phone,department_id) VALUES (89,'Jasmine','Hunt','jasmine.hunt@uga.edu','650.505.1876',23)")
conn.execute("INSERT INTO researchers (id,first_name,last_name,email,phone,department_id) VALUES (90,'Eliza','Black','eliza.black@uga.edu','650.505.2876',23)")
conn.execute("INSERT INTO researchers (id,first_name,last_name,email,phone,department_id) VALUES (91,'Lilly','Daniels','lilly.daniels@uga.edu','650.505.3876',23)")
conn.execute("INSERT INTO researchers (id,first_name,last_name,email,phone,department_id) VALUES (92,'Abigail','Palmer','abigail.palmer@uga.edu','650.505.4876',23)")
conn.execute("INSERT INTO researchers (id,first_name,last_name,email,phone,department_id) VALUES (93,'Georgia','Mills','georgia.mills@uga.edu','650.501.1876',24)")
conn.execute("INSERT INTO researchers (id,first_name,last_name,email,phone,department_id) VALUES (94,'Maisie','Nichols','maisie.nichols@uga.edu','650.501.2876',24)")
conn.execute("INSERT INTO researchers (id,first_name,last_name,email,phone,department_id) VALUES (95,'Eleanor','Grant','eleanor.grant@uga.edu','650.501.3876',24)")
conn.execute("INSERT INTO researchers (id,first_name,last_name,email,phone,department_id) VALUES (96,'Hannah','Knight','hannah.knight@uga.edu','650.501.4876',24)")
conn.execute("INSERT INTO researchers (id,first_name,last_name,email,phone,department_id) VALUES (97,'Harriet','Ferguson','harriet.ferguson@uga.edu','650.507.9811',25)")
conn.execute("INSERT INTO researchers (id,first_name,last_name,email,phone,department_id) VALUES (98,'Amber','Rose','amber.rose@uga.edu','650.507.9822',25)")
conn.execute("INSERT INTO researchers (id,first_name,last_name,email,phone,department_id) VALUES (99,'Bella','Stone','bella.stone@uga.edu','650.507.9833',25)")
conn.execute("INSERT INTO researchers (id,first_name,last_name,email,phone,department_id) VALUES (100,'Thea','Hawkins','thea.hawkins@uga.edu','650.507.9844',25)")
conn.execute("INSERT INTO researchers (id,first_name,last_name,email,phone,department_id) VALUES (81,'Lola','Ramos','lola.ramos@uga.edu','650.507.9876',21)")
conn.execute("INSERT INTO researchers (id,first_name,last_name,email,phone,department_id) VALUES (82,'Willow','Reyes','willow.reyes@uga.edu','650.507.9877',21)")
conn.execute("INSERT INTO researchers (id,first_name,last_name,email,phone,department_id) VALUES (83,'Ivy','Burns','ivy.burns@uga.edu','650.507.9878',21)")
conn.execute("INSERT INTO researchers (id,first_name,last_name,email,phone,department_id) VALUES (84,'Erin','Gordon','erin.gordon@uga.edu','650.507.9879',21)")
conn.execute("INSERT INTO researchers (id,first_name,last_name,email,phone,department_id) VALUES (33,'Reggie','Simmons','reggie.simmons@uga.edu','650.124.8234',22)")
conn.execute("INSERT INTO researchers (id,first_name,last_name,email,phone,department_id) VALUES (44,'Emily','Hamilton','emily.hamilton@uga.edu','650.121.2874',25)")
conn.execute("INSERT INTO researchers (id,first_name,last_name,email,phone,department_id) VALUES (43,'Olivia','Ford','olivia.ford@uga.edu','650.121.2994',25)")
conn.execute("INSERT INTO researchers (id,first_name,last_name,email,phone,department_id) VALUES (42,'Amelia','Myers','amelia.myers@uga.edu','650.121.8009',25)")
conn.execute("INSERT INTO researchers (id,first_name,last_name,email,phone,department_id) VALUES (41,'Connor','Hayes','connor.hayes@uga.edu','650.121.1834',24)")
conn.execute("INSERT INTO researchers (id,first_name,last_name,email,phone,department_id) VALUES (26,'Leon','Powell','leon.powell@uga.edu','650.124.1214',21)")
conn.execute("INSERT INTO researchers (id,first_name,last_name,email,phone,department_id) VALUES (27,'Kai','Long','kai.long@uga.edu','650.124.1224',21)")
conn.execute("INSERT INTO researchers (id,first_name,last_name,email,phone,department_id) VALUES (28,'Aaron','Patterson','aaron.patterson@uga.edu','650.124.1334',21)")
conn.execute("INSERT INTO researchers (id,first_name,last_name,email,phone,department_id) VALUES (29,'Roman','Hughes','roman.hughes@uga.edu','650.124.1434',21)")
conn.execute("INSERT INTO researchers (id,first_name,last_name,email,phone,department_id) VALUES (30,'Austin','Flores','austin.flores@uga.edu','650.124.5234',22)")
conn.execute("INSERT INTO researchers (id,first_name,last_name,email,phone,department_id) VALUES (31,'Ellis','Washington','ellis.washington@uga.edu','650.124.6234',22)")
conn.execute("INSERT INTO researchers (id,first_name,last_name,email,phone,department_id) VALUES (32,'Jamie','Butler','jamie.butler@uga.edu','650.124.7234',22)")
conn.execute("INSERT INTO researchers (id,first_name,last_name,email,phone,department_id) VALUES (45,'Isla','Graham','isla.graham@uga.edu','650.121.2004',25)")
conn.execute("INSERT INTO researchers (id,first_name,last_name,email,phone,department_id) VALUES (34,'Seth','Foster','seth.foster@uga.edu','650.127.1934',23)")
conn.execute("INSERT INTO researchers (id,first_name,last_name,email,phone,department_id) VALUES (35,'Carter','Gonzales','carter.gonzales@uga.edu','650.127.1834',23)")
conn.execute("INSERT INTO researchers (id,first_name,last_name,email,phone,department_id) VALUES (36,'Felix','Bryant','felix.bryant@uga.edu','650.127.1734',23)")
conn.execute("INSERT INTO researchers (id,first_name,last_name,email,phone,department_id) VALUES (37,'Ibrahim','Alexander','ibrahim.alexander@uga.edu','650.127.1634',23)")
conn.execute("INSERT INTO researchers (id,first_name,last_name,email,phone,department_id) VALUES (38,'Sonny','Russell','sonny.russell@uga.edu','650.121.1234',24)")
conn.execute("INSERT INTO researchers (id,first_name,last_name,email,phone,department_id) VALUES (39,'Kian','Griffin','kian.griffin@uga.edu','650.121.2034',24)")
conn.execute("INSERT INTO researchers (id,first_name,last_name,email,phone,department_id) VALUES (40,'Caleb','Diaz','caleb.diaz@uga.edu','650.121.2019',24)")
conn.execute("INSERT INTO researchers (id,first_name,last_name,email,phone,department_id) VALUES (25,'Ronnie','Perry','ronnie.perry@uga.edu','650.123.5234',1)")
conn.execute("INSERT INTO researchers (id,first_name,last_name,email,phone,department_id) VALUES (24,'Callum','Jenkins','callum.jenkins@uga.edu','650.123.4234',1)")
conn.execute("INSERT INTO researchers (id,first_name,last_name,email,phone,department_id) VALUES (23,'Jackson','Coleman','jackson.coleman@uga.edu','650.123.3234',1)")
conn.execute("INSERT INTO researchers (id,first_name,last_name,email,phone,department_id) VALUES (22,'Liam','Henderson','liam.henderson@uga.edu','650.123.2234',1)")
conn.execute("INSERT INTO researchers (id,first_name,last_name,email,phone,department_id) VALUES (21,'Jaxon','Ross','jaxon.ross@uga.edu','650.123.1234',1)")

df = pd.read_sql_query("select * from researchers;", conn)
print("Researchers\n", df)


Researchers
       id first_name  last_name                     email         phone  \
0      0     Javier  Rodriguez  javier.rodriguez@uga.edu  686.300.9440   
1    107     Summer      Payne      summer.payne@uga.edu  515.123.8181   
2    106       Rose   Stephens     rose.stephens@uga.edu  515.123.8080   
3    101  Annabelle       Dunn    annabelle.dunn@uga.edu  515.123.4444   
4      1      Tommy     Bailey      tommy.bailey@uga.edu  515.123.4567   
..   ...        ...        ...                       ...           ...   
103   25     Ronnie      Perry      ronnie.perry@uga.edu  650.123.5234   
104   24     Callum    Jenkins    callum.jenkins@uga.edu  650.123.4234   
105   23    Jackson    Coleman   jackson.coleman@uga.edu  650.123.3234   
106   22       Liam  Henderson    liam.henderson@uga.edu  650.123.2234   
107   21      Jaxon       Ross        jaxon.ross@uga.edu  650.123.1234   

     department_id  
0              0.0  
1            106.0  
2              2.0  
3             

In [10]:
conn.execute("INSERT INTO projects (id,name,status,start_month,start_year) VALUES (107,'Project1','Finished','October',2017)")
conn.execute("INSERT INTO projects (id,name,status,start_month,start_year) VALUES (106,'Project2','Finished','August',2018)")
conn.execute("INSERT INTO projects (id,name,status,start_month,start_year) VALUES (101,'Project3','Finished','July',2018)")
conn.execute("INSERT INTO projects (id,name,status,start_month,start_year) VALUES (1,'Project4','Finished','July',2015)")
conn.execute("INSERT INTO projects (id,name,status,start_month,start_year) VALUES (3,'Project5','Finished','July',2018)")
conn.execute("INSERT INTO projects (id,name,status,start_month,start_year) VALUES (2,'Project6','Finished','March',2018)")
conn.execute("INSERT INTO projects (id,name,status,start_month,start_year) VALUES (11,'Project7','In Execution','December',2019)")
conn.execute("INSERT INTO projects (id,name,status,start_month,start_year) VALUES (10,'Project8','Finished','March',2018)")
conn.execute("INSERT INTO projects (id,name,status,start_month,start_year) VALUES (14,'Project9','Finished','August',2018)")
conn.execute("INSERT INTO projects (id,name,status,start_month,start_year) VALUES (12,'Project10','Finished','July',2018)")
conn.execute("INSERT INTO projects (id,name,status,start_month,start_year) VALUES (13,'Project11','Finished','July',2018)")
conn.execute("INSERT INTO projects (id,name,status,start_month,start_year) VALUES (9,'Project12','Finished','November',2017)")
conn.execute("INSERT INTO projects (id,name,status,start_month,start_year) VALUES (104,'Project13','Finished','December',2016)")
conn.execute("INSERT INTO projects (id,name,status,start_month,start_year) VALUES (4,'Project14','Finished','July',2017)")
conn.execute("INSERT INTO projects (id,name,status,start_month,start_year) VALUES (5,'Project15','Finished','July',2016)")
conn.execute("INSERT INTO projects (id,name,status,start_month,start_year) VALUES (8,'Project16','In Execution','June',2019)")
conn.execute("INSERT INTO projects (id,name,status,start_month,start_year) VALUES (7,'Project17','Finished','July',2017)")
conn.execute("INSERT INTO projects (id,name,status,start_month,start_year) VALUES (6,'Project18','Finished','December',2018)")
conn.execute("INSERT INTO projects (id,name,status,start_month,start_year) VALUES (102,'Project19','Finished','January',2017)")
conn.execute("INSERT INTO projects (id,name,status,start_month,start_year) VALUES (103,'Project20','Finished','August',2017)")
conn.execute("INSERT INTO projects (id,name,status,start_month,start_year) VALUES (105,'Project21','Finished','June',2018)")
conn.execute("INSERT INTO projects (id,name,status,start_month,start_year) VALUES (17,'Project22','Finished','September',2017)")
conn.execute("INSERT INTO projects (id,name,status,start_month,start_year) VALUES (16,'Project23','Finished','September',2015)")
conn.execute("INSERT INTO projects (id,name,status,start_month,start_year) VALUES (18,'Project24','Finished','July',2015)")
conn.execute("INSERT INTO projects (id,name,status,start_month,start_year) VALUES (19,'Project25','Finished','August',2017)")
conn.execute("INSERT INTO projects (id,name,status,start_month,start_year) VALUES (20,'Project26','Finished','September',2017)")
conn.execute("INSERT INTO projects (id,name,status,start_month,start_year) VALUES (15,'Project27','Finished','August',2018)")
conn.execute("INSERT INTO projects (id,name,status,start_month,start_year) VALUES (49,'Project28','Finished','July',2017)")
conn.execute("INSERT INTO projects (id,name,status,start_month,start_year) VALUES (48,'Project29','Finished','July',2016)")
conn.execute("INSERT INTO projects (id,name,status,start_month,start_year) VALUES (47,'Project30','Finished','October',2017)")
conn.execute("INSERT INTO projects (id,name,status,start_month,start_year) VALUES (46,'Project31','Finished','November',2017)")
conn.execute("INSERT INTO projects (id,name,status,start_month,start_year) VALUES (50,'Project32','Finished','November',2016)")
conn.execute("INSERT INTO projects (id,name,status,start_month,start_year) VALUES (56,'Project33','Finished','July',2016)")
conn.execute("INSERT INTO projects (id,name,status,start_month,start_year) VALUES (57,'Project34','Finished','October',2015)")
conn.execute("INSERT INTO projects (id,name,status,start_month,start_year) VALUES (58,'Project35','Finished','May',2016)")
conn.execute("INSERT INTO projects (id,name,status,start_month,start_year) VALUES (59,'Project36','Finished','August',2018)")
conn.execute("INSERT INTO projects (id,name,status,start_month,start_year) VALUES (60,'Project37','Finished','July',2017)")
conn.execute("INSERT INTO projects (id,name,status,start_month,start_year) VALUES (61,'Project38','Finished','September',2015)")
conn.execute("INSERT INTO projects (id,name,status,start_month,start_year) VALUES (62,'Project39','Finished','May',2017)")
conn.execute("INSERT INTO projects (id,name,status,start_month,start_year) VALUES (80,'Project40','In Execution','February',2019)")
conn.execute("INSERT INTO projects (id,name,status,start_month,start_year) VALUES (64,'Project41','Finished','July',2016)")
conn.execute("INSERT INTO projects (id,name,status,start_month,start_year) VALUES (65,'Project42','Finished','May',2018)")
conn.execute("INSERT INTO projects (id,name,status,start_month,start_year) VALUES (66,'Project43','In Execution','July',2019)")
conn.execute("INSERT INTO projects (id,name,status,start_month,start_year) VALUES (67,'Project44','In Execution','September',2019)")
conn.execute("INSERT INTO projects (id,name,status,start_month,start_year) VALUES (68,'Project45','Finished','August',2018)")
conn.execute("INSERT INTO projects (id,name,status,start_month,start_year) VALUES (69,'Project46','Finished','July',2015)")
conn.execute("INSERT INTO projects (id,name,status,start_month,start_year) VALUES (70,'Project47','Finished','April',2017)")
conn.execute("INSERT INTO projects (id,name,status,start_month,start_year) VALUES (71,'Project48','In Execution','May',2019)")
conn.execute("INSERT INTO projects (id,name,status,start_month,start_year) VALUES (72,'Project49','In Execution','November',2019)")
conn.execute("INSERT INTO projects (id,name,status,start_month,start_year) VALUES (73,'Project50','In Execution','April',2019)")
conn.execute("INSERT INTO projects (id,name,status,start_month,start_year) VALUES (75,'Project51','Finished','May',2016)")
conn.execute("INSERT INTO projects (id,name,status,start_month,start_year) VALUES (76,'Project52','Finished','December',2016)")
conn.execute("INSERT INTO projects (id,name,status,start_month,start_year) VALUES (77,'Project53','Finished','September',2015)")
conn.execute("INSERT INTO projects (id,name,status,start_month,start_year) VALUES (78,'Project54','Finished','July',2015)")
conn.execute("INSERT INTO projects (id,name,status,start_month,start_year) VALUES (79,'Project55','Finished','March',2017)")
conn.execute("INSERT INTO projects (id,name,status,start_month,start_year) VALUES (55,'Project56','Finished','July',2018)")
conn.execute("INSERT INTO projects (id,name,status,start_month,start_year) VALUES (54,'Project57','Finished','November',2018)")
conn.execute("INSERT INTO projects (id,name,status,start_month,start_year) VALUES (53,'Project58','In Execution','August',2019)")
conn.execute("INSERT INTO projects (id,name,status,start_month,start_year) VALUES (52,'Project59','Finished','September',2018)")
conn.execute("INSERT INTO projects (id,name,status,start_month,start_year) VALUES (51,'Project60','Finished','July',2016)")
conn.execute("INSERT INTO projects (id,name,status,start_month,start_year) VALUES (63,'Project61','Finished','April',2015)")
conn.execute("INSERT INTO projects (id,name,status,start_month,start_year) VALUES (85,'Project62','Finished','April',2018)")
conn.execute("INSERT INTO projects (id,name,status,start_month,start_year) VALUES (86,'Project63','Finished','April',2015)")
conn.execute("INSERT INTO projects (id,name,status,start_month,start_year) VALUES (87,'Project64','Finished','July',2015)")
conn.execute("INSERT INTO projects (id,name,status,start_month,start_year) VALUES (88,'Project65','Finished','February',2017)")
conn.execute("INSERT INTO projects (id,name,status,start_month,start_year) VALUES (89,'Project66','Finished','February',2015)")
conn.execute("INSERT INTO projects (id,name,status,start_month,start_year) VALUES (90,'Project67','In Execution','July',2019)")
conn.execute("INSERT INTO projects (id,name,status,start_month,start_year) VALUES (91,'Project68','Finished','September',2015)")
conn.execute("INSERT INTO projects (id,name,status,start_month,start_year) VALUES (92,'Project69','Finished','July',2016)")
conn.execute("INSERT INTO projects (id,name,status,start_month,start_year) VALUES (93,'Project70','Finished','April',2017)")
conn.execute("INSERT INTO projects (id,name,status,start_month,start_year) VALUES (94,'Project71','In Execution','March',2019)")
conn.execute("INSERT INTO projects (id,name,status,start_month,start_year) VALUES (95,'Project72','Finished','March',2018)")
conn.execute("INSERT INTO projects (id,name,status,start_month,start_year) VALUES (96,'Project73','Finished','July',2018)")
conn.execute("INSERT INTO projects (id,name,status,start_month,start_year) VALUES (97,'Project74','Finished','September',2018)")
conn.execute("INSERT INTO projects (id,name,status,start_month,start_year) VALUES (98,'Project75','Finished','July',2018)")
conn.execute("INSERT INTO projects (id,name,status,start_month,start_year) VALUES (99,'Project76','In Execution','August',2019)")
conn.execute("INSERT INTO projects (id,name,status,start_month,start_year) VALUES (100,'Project77','Finished','March',2015)")
conn.execute("INSERT INTO projects (id,name,status,start_month,start_year) VALUES (81,'Project78','Finished','March',2016)")
conn.execute("INSERT INTO projects (id,name,status,start_month,start_year) VALUES (82,'Project79','In Execution','September',2019)")
conn.execute("INSERT INTO projects (id,name,status,start_month,start_year) VALUES (83,'Project80','Finished','November',2016)")
conn.execute("INSERT INTO projects (id,name,status,start_month,start_year) VALUES (84,'Project81','In Execution','August',2019)")
conn.execute("INSERT INTO projects (id,name,status,start_month,start_year) VALUES (33,'Project82','In Execution','July',2019)")
conn.execute("INSERT INTO projects (id,name,status,start_month,start_year) VALUES (44,'Project83','In Execution','November',2019)")
conn.execute("INSERT INTO projects (id,name,status,start_month,start_year) VALUES (43,'Project84','In Execution','April',2019)")
conn.execute("INSERT INTO projects (id,name,status,start_month,start_year) VALUES (42,'Project85','Finished','April',2018)")
conn.execute("INSERT INTO projects (id,name,status,start_month,start_year) VALUES (41,'Project86','In Execution','January',2019)")
conn.execute("INSERT INTO projects (id,name,status,start_month,start_year) VALUES (26,'Project87','Finished','January',2016)")
conn.execute("INSERT INTO projects (id,name,status,start_month,start_year) VALUES (27,'Project88','Finished','December',2015)")
conn.execute("INSERT INTO projects (id,name,status,start_month,start_year) VALUES (28,'Project89','Finished','July',2017)")
conn.execute("INSERT INTO projects (id,name,status,start_month,start_year) VALUES (29,'Project90','Finished','April',2018)")
conn.execute("INSERT INTO projects (id,name,status,start_month,start_year) VALUES (30,'Project91','Finished','September',2015)")
conn.execute("INSERT INTO projects (id,name,status,start_month,start_year) VALUES (31,'Project92','In Execution','October',2019)")
conn.execute("INSERT INTO projects (id,name,status,start_month,start_year) VALUES (32,'Project93','In Execution','July',2019)")
conn.execute("INSERT INTO projects (id,name,status,start_month,start_year) VALUES (45,'Project94','Finished','July',2018)")
conn.execute("INSERT INTO projects (id,name,status,start_month,start_year) VALUES (34,'Project95','Finished','October',2015)")
conn.execute("INSERT INTO projects (id,name,status,start_month,start_year) VALUES (35,'Project96','Finished','July',2018)")
conn.execute("INSERT INTO projects (id,name,status,start_month,start_year) VALUES (36,'Project97','Finished','July',2015)")
conn.execute("INSERT INTO projects (id,name,status,start_month,start_year) VALUES (37,'Project98','Finished','November',2016)")
conn.execute("INSERT INTO projects (id,name,status,start_month,start_year) VALUES (38,'Project99','Finished','March',2017)")
conn.execute("INSERT INTO projects (id,name,status,start_month,start_year) VALUES (39,'Project100','Finished','November',2017)")
conn.execute("INSERT INTO projects (id,name,status,start_month,start_year) VALUES (40,'Project101','Finished','July',2018)")
conn.execute("INSERT INTO projects (id,name,status,start_month,start_year) VALUES (25,'Project102','Finished','April',2015)")
conn.execute("INSERT INTO projects (id,name,status,start_month,start_year) VALUES (24,'Project103','Finished','February',2017)")
conn.execute("INSERT INTO projects (id,name,status,start_month,start_year) VALUES (23,'Project104','In Execution','October',2019)")
conn.execute("INSERT INTO projects (id,name,status,start_month,start_year) VALUES (22,'Project105','Finished','April',2015)")
conn.execute("INSERT INTO projects (id,name,status,start_month,start_year) VALUES (21,'Project106','Finished','August',2016)")

df = pd.read_sql_query("select * from projects;", conn)
print("Projects\n", df)

Projects
       id        name        status start_month  start_year
0      0    Project0        Active     January        2020
1    107    Project1      Finished     October        2017
2    106    Project2      Finished      August        2018
3    101    Project3      Finished        July        2018
4      1    Project4      Finished        July        2015
..   ...         ...           ...         ...         ...
102   25  Project102      Finished       April        2015
103   24  Project103      Finished    February        2017
104   23  Project104  In Execution     October        2019
105   22  Project105      Finished       April        2015
106   21  Project106      Finished      August        2016

[107 rows x 5 columns]


In [211]:
#Evaluate Database's consistency
conn = sql.connect("Assignment2.db")
cur = conn.cursor()
conn.execute("PRAGMA integrity_check")

df = pd.read_sql_query("SELECT * FROM departments;", conn)
print("\n",df)
#cur.execute("INSERT INTO projects VALUES (1,'Active','February',2019)")
cur.execute("INSERT INTO departments VALUES (6,'Romance Languages','Athens')")
cur.execute("UPDATE departments SET name = 'Mathematics' WHERE id = 2;")
cur.execute("INSERT INTO departments VALUES (5,'Physics','Tifton')")
df = pd.read_sql_query("SELECT * FROM departments;", conn)
print("\n",df)

df = pd.read_sql_query("SELECT * FROM researchers;", conn)
print("\n",df)
cur.execute("UPDATE researchers SET phone = '786-003344' where department_id = 1;")
df = pd.read_sql_query("SELECT * FROM researchers;", conn)
print("\n",df)

#Query Database
conn = sql.connect("Assignment2.db")
cur = conn.cursor()
conn.execute("PRAGMA integrity_check")

df = pd.read_sql_query("SELECT first_name,last_name, department_id FROM researchers;", conn)
print("\n",df)
df = pd.read_sql_query("SELECT researchers.first_name, researchers.last_name, departments.name FROM researchers INNER JOIN departments ON researchers.department_id=departments.id;",conn)
print("\n",df)

conn.commit();

cur.close()
conn.close()



    id              name     campus
0   1       Engineering     Athens
1   2                       Athens
2   3  Plant production     Tifton
3   4          Genetics  Riverbend

    id               name     campus
0   1        Engineering     Athens
1   2        Mathematics     Athens
2   3   Plant production     Tifton
3   4           Genetics  Riverbend
4   6  Romance Languages     Athens
5   5            Physics     Tifton

    id first_name  last_name       phone  department_id
0   1     Javier  Rodriguez  686-300944              1
1   2     Nicole     Garcia  706-300944              1
2   3      David             706-443900              2
3   4     Sierra  Casnettie  709-555678              1

    id first_name  last_name       phone  department_id
0   1     Javier  Rodriguez  786-003344              1
1   2     Nicole     Garcia  786-003344              1
2   3      David             706-443900              2
3   4     Sierra  Casnettie  786-003344              1

   first_name 